In [ ]:
import numpy as np
import re, json, csv
import torch
from matplotlib import pyplot as plt
from pymongo import MongoClient
from tqdm.notebook import tqdm
from pprint import PrettyPrinter
from typing import Union, Iterable
from bson.objectid import ObjectId
from torchmetrics.text.bert import BERTScore
from torchmetrics.text.rouge import ROUGEScore
from torchmetrics.text.bleu import BLEUScore
from transformers import AutoModelForSeq2SeqLM, AutoModelForCausalLM, AutoConfig, AutoTokenizer, pipeline, set_seed
from torchmetrics.functional import pairwise_cosine_similarity
from torchmetrics.functional.text.bert import _get_precision_recall_f1
from functools import partial
from collections import Counter, defaultdict
from itertools import chain
from math import log
from multiprocessing import Pool
from typing import List
from pymongo import MongoClient
from typing import Set

pprint = PrettyPrinter().pprint

In [ ]:
DEVICE = "cpu"

In [ ]:
db = MongoClient("mongodb://localhost:27017")["cuda_snippets"]
train_db = db["train"]
validation_db = db["validation"]

### Get model

### Calculate ROUGE

In [ ]:
def compute_rouge_score(target_sentences : Iterable[str], pred_senteces : Iterable[str], tokenizer):
    tokenizer_fn = lambda x: tokenizer.convert_ids_to_tokens(tokenizer.encode(x))
    rouge_metric = ROUGEScore(tokenizer=tokenizer_fn)
    # rouge_metric.update(pred_senteces, target_sentences)
    return rouge_metric(pred_senteces, target_sentences)

### Calculate BLEU

In [ ]:
def compute_bleu_score(target_sentences : Iterable[str], pred_senteces : Iterable[str], tokenizer):
    bleu_metric = BLEUScore(tokenizer=tokenizer)
    bleu_metric.update(pred_senteces, [[s] for s in target_sentences])
    return bleu_metric.compute()

### Calculate BERT score

In [ ]:
def process(a, tokenizer=None):
    if tokenizer is not None:
        a = tokenizer.encode(a, max_length=512, truncation=True, add_special_tokens=False)
    return set(a)

def get_idf_dict(arr, tokenizer, nthreads=4):
    idf_count = Counter()
    num_docs = len(arr)

    process_partial = partial(process, tokenizer=tokenizer)

    if nthreads > 0:
        with Pool(nthreads) as p:
            idf_count.update(chain.from_iterable(p.map(process_partial, arr)))
    else:
        idf_count.update(chain.from_iterable(map(process_partial, arr)))

    idf_dict = defaultdict(lambda: log((num_docs + 1) / (1)))
    idf_dict.update(
        {idx: log((num_docs + 1) / (c + 1)) for (idx, c) in idf_count.items()}
    )
    return idf_dict

class TextDataset:
    def __init__(self, target_sentences, pred_sentences) -> None:
        self.target_sentences = target_sentences
        self.pred_sentences = pred_sentences
    
    def __len__(self):
        return len(self.target_sentences)
    
    def __getitem__(self, i):
        return self.target_sentences[i], self.pred_sentences[i]
    
class CollateFn:
    def __init__(self, tokenizer) -> None:
        self.tokenizer = tokenizer
    
    def __call__(self, samples : list):
        targets, preds = zip(*samples)
        
        targets_pt = self.tokenizer(targets, max_length=512, truncation=True, return_tensors="pt", add_special_tokens=False, padding=True)
        preds_pt = self.tokenizer(preds, max_length=512, truncation=True, return_tensors="pt", add_special_tokens=False, padding=True)

        return (targets_pt, targets), (preds_pt, preds)

def compute_bert_score(target_sentences : Iterable[str], pred_sentences : Iterable[str], tokenizer, embedd_layer : torch.nn.Embedding, idf_scores : dict = None, batch_size=64):

    if idf_scores is None:
        idf_scores = get_idf_dict(target_sentences, tokenizer)
    precs = []
    recals = []
    f1s = []
    
    dataset = TextDataset(target_sentences, pred_sentences)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=CollateFn(tokenizer))
    
    for i, ((target_pt, target_str), (pred_pt, pred_str)) in enumerate(tqdm(dataloader)):

        pred_embedd = embedd_layer(pred_pt["input_ids"]).view((pred_pt["input_ids"].size(0),1,pred_pt["input_ids"].size(1), -1))
        target_embedd = embedd_layer(target_pt["input_ids"]).view((target_pt["input_ids"].size(0),1,target_pt["input_ids"].size(1), -1))
        
        pred_idfs = [tokenizer.encode(sentence, max_length=512, add_special_tokens=False, truncation=True, return_tensors="pt")[0] for sentence in pred_str]
        pred_idfs = [torch.Tensor([idf_scores.get(token, 0) for token in sentence]) for sentence in pred_idfs]
        pred_idfs = torch.nn.utils.rnn.pad_sequence(pred_idfs, batch_first=True, padding_value=0)
        
        target_idfs = [tokenizer.encode(sentence, max_length=512, add_special_tokens=False, truncation=True, return_tensors="pt")[0] for sentence in target_str]
        target_idfs = [torch.Tensor([idf_scores.get(token, 0) for token in sentence]) for sentence in target_idfs]
        target_idfs = torch.nn.utils.rnn.pad_sequence(target_idfs, batch_first=True, padding_value=0)

        # pred_idfs = torch.Tensor(
        #     [[idf_scores.get(token, 0) for token in sentence] for sentence in pred_str]
        # )
        
        # target_idfs = torch.Tensor(
        #     [[idf_scores.get(token, 0) for token in sentence] for sentence in target_str]
        # )
        
        # cos_sims = pairwise_cosine_similarity(target_embedd, pred_embedd)
        
        # max_sims = torch.max(cos_sims, dim=1)[0]
        # sentence_scores.append(sum(max_sims*target_idf)/sum(target_idf))
        
        prec, res, f1 = _get_precision_recall_f1(pred_embedd, target_embedd, pred_idfs, target_idfs)
        precs.append(torch.mean(prec))
        recals.append(torch.mean(res))
        f1s.append(torch.mean(f1))
        
    return  sum(precs) / len(precs), \
            sum(recals) / len(recals), \
            sum(f1s) / len(f1s)
        

In [ ]:
def update(self, preds: List[str], targets: List[str]) -> None:
        """Store predictions/references for computing BERT scores.

        It is necessary to store sentences in a tokenized form to ensure the DDP mode working.
        """
        
        preds_tokenized_data = self.tokenizer(preds, max_length=1024, truncation=True, return_tensors="pt", padding=True)
        preds_dict = {"input_ids": preds_tokenized_data["input_ids"], "attention_mask": preds_tokenized_data["attention_mask"]}
        target_tokenized_data = self.tokenizer(targets, max_length=1024, truncation=True, return_tensors="pt", padding=True)
        target_dict = {"input_ids": target_tokenized_data["input_ids"], "attention_mask": target_tokenized_data["attention_mask"]}

        self.preds_input_ids.append(preds_dict["input_ids"])
        self.preds_attention_mask.append(preds_dict["attention_mask"])
        self.target_input_ids.append(target_dict["input_ids"])
        self.target_attention_mask.append(target_dict["attention_mask"])
        
def compute_bert_score2(target_sentences : Iterable[str], pred_senteces : Iterable[str], tokenizer, model : torch.nn.Module, *args, **kwargs):
    BERTScore.update = update
    user_forward_fn = lambda model, d: model.get_input_embeddings()(d["input_ids"])
    bert_metric = BERTScore(model=model, user_tokenizer=tokenizer, user_forward_fn=user_forward_fn, device=DEVICE, verbose=True, num_layers=0, max_length=1024)
    # bert_metric.update = update
    output = {}
    for key, vals in bert_metric(preds=pred_senteces, targets=target_sentences).items():
        output[key] = float(np.mean(vals))
    
    return output
#     bert_metric.update(bert_metric, pred_senteces, target_sentences)
#     return bert_metric.compute()

In [ ]:
def get_kernel_prefixes(kernel : str) -> Set[str]:
    prefixes = set()
    one_line_kernel = kernel.replace("\n", " ")
    cuda_header_prefix_re = re.compile("__(host|global|device)__")
    
    prefixes.update(cuda_header_prefix_re.findall(one_line_kernel))
    return prefixes
        

In [ ]:
def compute_metrics(sources : List[str], targets : List[str], preds : List[str], tokenizer, model : torch.nn.Module, used_ratio : float = 1.0):
    assert used_ratio > 0 and used_ratio <= 1
    assert len(targets) == len(preds) and len(targets) == len(sources)
    
    samples = list(zip(targets, preds))
    np.random.shuffle(samples)
    n = round(len(samples) * used_ratio)
        
    metrics_d = {
        "device" : {},
        "host" : {},
        "global" : {}
    }
    
    device_sentences = []
    host_sentences = []
    global_sentences = []
    
    pb = tqdm(zip(sources, targets, preds))
    i = 0
    for src, target, pred in pb:

        found = False
        
        if found := (src.find("__device__") != -1):
            device_sentences.append((target, pred))
        if found := (src.find("__host__") != -1):
            host_sentences.append((target, pred))
        if found := (src.find("__global__") != -1):
            global_sentences.append((target, pred))
        
        if not found:
            continue
        
        i += 1
        if i >= n:
            break
    
    if len(device_sentences) == 0:
        print("WARNING: no device sample")
        # metrics_d["__device__"] = {"rouge" : 0, "bleu" : 0, "bert" : 0}
    else:
        device_targets, device_preds = zip(*device_sentences)
        metrics_d["device"] = {
            "rouge" : compute_rouge_score(device_targets, device_preds, tokenizer),
            "bleu" : compute_bleu_score(device_targets, device_preds, tokenizer),
            "bert" : compute_bert_score2(device_targets, device_preds, tokenizer, model)
        }
        print(f"__device__ ({len(device_sentences)})")
        pprint(metrics_d["device"])
        
    if len(host_sentences) == 0:
        print("WARNING: no host samples")
        # metrics_d["__host__"] = {"rouge" : 0, "bleu" : 0, "bert" : 0}
    else:
        host_targets, host_preds = zip(*host_sentences)
        metrics_d["host"] = {
            "rouge" : compute_rouge_score(host_targets, host_preds, tokenizer),
            "bleu" : compute_bleu_score(host_targets, host_preds, tokenizer),
            "bert" : compute_bert_score2(host_targets, host_preds, tokenizer, model)
        }
        print(f"__host__ ({len(host_sentences)})")
        pprint(metrics_d["host"])
        
    if len(global_sentences) == 0:
        print("WARNING: no global samples")
        # metrics_d["__global__"] = {"rouge" : 0, "bleu" : 0, "bert" : 0}
    else:
        global_targets, global_preds = zip(*global_sentences)
        metrics_d["global"] = {
            "rouge" : compute_rouge_score(global_targets, global_preds, tokenizer),
            "bleu" : compute_bleu_score(global_targets, global_preds, tokenizer),
            "bert" : compute_bert_score2(global_targets, global_preds, tokenizer, model)
        }
        print(f"__global__ ({len(global_sentences)})")
        pprint(metrics_d["global"])
        
    # Weights 
    total = len(device_sentences) + len(host_sentences) + len(global_sentences)
    d_w = len(device_sentences) / total
    h_w = len(host_sentences) / total
    g_w = len(global_sentences) / total
    
    def calculate_total_metric(ds : List[dict], ws : List[float]):        
        def calculate(ds : List[dict], ws : List[float], key : str):
            r = 0
            for d, w in zip(ds, ws):
                if d is None:
                    continue
                
                r += d.get(key, 0) * w
            return r
        
        total_rouge = {}
        
        for d in ds:
            if d is None:
                continue
            for key in d.keys():
                if key in total_rouge:
                    continue
                
                total_rouge[key] = calculate(ds, ws, key)
        return total_rouge
    
    metrics_d["total"] = {"rouge" : {}, "bleu" : -1, "bert" : {}}
    metrics_d["total"]["rouge"] = calculate_total_metric(
        [metrics_d["device"].get("rouge"), metrics_d["host"].get("rouge"), metrics_d["global"].get("rouge")],
        [d_w, h_w, g_w]
    )
    
    metrics_d["total"]["bleu"] = metrics_d["device"].get("bleu",0)*d_w + metrics_d["host"].get("bleu",0)*h_w + metrics_d["global"].get("bleu",0)*g_w

    
    metrics_d["total"]["bert"] = calculate_total_metric(
        [metrics_d["device"].get("bert"), metrics_d["host"].get("bert"), metrics_d["global"].get("bert")],
        [d_w, h_w, g_w]
    )
    
    print("total")
    pprint(metrics_d["total"])
    
    # rouge = compute_rouge_score(targets, preds, tokenizer)
    # pprint({"rouge" : rouge})

    # bleu = compute_bleu_score(targets, preds, tokenizer)
    # pprint({"bleu" : bleu})

    # bert = compute_bert_score2(targets, preds, tokenizer, model)
    # pprint({"bert" : bert})
    
    return metrics_d

# GPT2

In [ ]:
def get_gpt2_model(model_name : str, model_path : str, tokenizer):

    configuration = AutoConfig.from_pretrained(model_name)

    model = AutoModelForCausalLM.from_config(configuration).to(DEVICE)
    # model.resize_token_embeddings(len(tokenizer))
    model_dict = torch.load(model_path, map_location="cpu")
    model.load_state_dict(model_dict["model_dict"])
    
    return model, model_dict

In [ ]:
model_name = "distilgpt2"
model_path = "../../../models/gpt2/gpt2.evaluated.pt"
tokenizer_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, use_fast=False, model_max_length=700, padding_side = "left", padding=True, truncation=True,)
tokenizer.padding_side = "left"
tokenizer.add_special_tokens({
    "pad_token" : "<pad>"
})
model, model_d = get_gpt2_model(model_name, model_path, tokenizer)


In [ ]:
used_ratio = 0.001
metrics = compute_metrics(model_d.get("source_sentences"), model_d.get("target_sentences"), model_d.get("predic_sentences"), tokenizer, model, used_ratio)

In [ ]:
loss_list = model_d.get("loss_list")
plt.figure()
plt.title("GPT2 loss")
plt.ylabel("Cross-entropy loss")
plt.xlabel("Epoch")
plt.plot(loss_list)
plt.show()

print(f"Epochs: {len(loss_list)}")
print(f"Max: {max(loss_list)}, epoch: {np.argmax(loss_list)}")
print(f"Min: {min(loss_list)}, epoch: {np.argmin(loss_list)}")

### Average lengths

In [ ]:
tgr_ls = 0
tgr_cs = 0
prd_ls = 0
prd_cs = 0
s = len(model_d.get("source_sentences"))
for tgr, prd in tqdm(zip(model_d.get("source_sentences"), model_d.get("target_sentences"))):
    # tgr = tgr.replace(";", ";\n").replace("{", "{\n").replace("}", "}\n").replace("int", "\nint").replace("float", "\nfloat")
    # prd = prd.replace(";", ";\n").replace("{", "{\n").replace("}", "}\n").replace("int", "\nint").replace("float", "\nfloat")
    
    tgr_ls += tgr.count("\n")
    prd_ls += prd.count("\n")
    
    tgr_cs += len(tgr)
    prd_cs += len(prd)
    
print("Average target lines per sample: {:.3f}".format(tgr_ls/s))
print("Average predic lines per sample: {:.3f}".format(prd_ls/s))
print("Average target sample size: {:.3f}".format(tgr_cs/s))
print("Average predic sample size: {:.3f}".format(prd_cs/s))


### Examples

In [ ]:
# Generate random example from evaluation
samples = list(zip(model_d.get("source_sentences"), model_d.get("target_sentences"), model_d.get("predic_sentences")))
np.random.shuffle(samples)
src_l, tgr_l, prd_l = zip(*samples)
for src, tgr, prd in zip(src_l, tgr_l, prd_l):
    
    if len(tgr) > 300:
        continue
    
    print("SOURCE:")
    print(src)
    print("\nTARGET:")
    print(tgr)
    print("\nPREDICT:")
    print(prd)
    print("#--------------------------#")
    break

In [ ]:
prompt = """
// Kernel for dividing by two
__device__ float divideByTwo(float v)
""".strip()

target = """
{
    return v / 2;
}
""".strip()

In [ ]:
from sklearn.linear_model import LinearRegression 
from scipy.stats import pearsonr

def scatter_hist(x, y, ax, ax_histx, ax_histy, title):
    # no labels
    ax_histx.tick_params(axis="x", labelbottom=False)
    ax_histy.tick_params(axis="y", labelleft=False)

    # the scatter plot:
    ax.scatter(x, y)
    
    regressor = LinearRegression() 
    regressor.fit(np.array(x).reshape((len(x),1)), y)
    p_y = regressor.predict(np.array(x).reshape((len(x),1)))
    corr, _ = pearsonr(x, y)
    ax.plot(x, p_y, linewidth=0.75, color="orange")
    ax.text(max(x)*0.7, max(p_y)*1.1, f"Pear. corr={corr:.2f}", weight='bold')

    # now determine nice limits by hand:
    binwidth = 0.25
    
    xmax = np.max(np.abs(x))
    xlim = (int(xmax/binwidth) + 1) * binwidth
    xbins = np.linspace(-xlim, xlim + binwidth, 50)
    
    ymax = np.max(np.abs(y))
    ylim = (int(ymax/binwidth) + 1) * binwidth
    ybins = np.linspace(-ylim, ylim + binwidth, 50)

    ax_histx.hist(x, bins=xbins)
    ax_histx.set_title(title)
    ax_histy.hist(y, bins=ybins, orientation='horizontal')

def plot_scatter_with_dist(x, y, title, xlabel, ylabel):
    x = np.array(x)
    y = np.array(y)
    
    # Start with a square Figure.
    fig = plt.figure(figsize=(6, 6))
    gs = fig.add_gridspec(2, 2,  width_ratios=(4, 1), height_ratios=(1, 4),
                        left=0.1, right=0.9, bottom=0.1, top=0.9,
                        wspace=0.05, hspace=0.05)
    # Create the Axes.
    ax = fig.add_subplot(gs[1, 0])
    ax_histx = fig.add_subplot(gs[0, 0], sharex=ax)
    ax_histy = fig.add_subplot(gs[1, 1], sharey=ax)
    
    min_x = np.quantile(x, 0.05)
    max_x = np.quantile(x, 0.95)
    min_y = np.quantile(y, 0.05)
    max_y = np.quantile(y, 0.95)
    
    x_mask = ((x > min_x) & (x < max_x))
    x = x[x_mask]
    y = y[x_mask]
    y_mask = ((y > min_y) & (y < max_y))
    x = x[y_mask]
    y = y[y_mask]
    
    # Draw the scatter plot and marginals.
    scatter_hist(x, y, ax, ax_histx, ax_histy, title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    
    ax.set_xlim(min_x*0.9, max_x*1.1)
    ax.set_ylim(min_y*0.9, max_y*1.1)
    plt.show()

In [ ]:
prd_sample_len = []
tgr_sample_len = []

for prd, tgr in zip(model_d["predic_sentences"], model_d["target_sentences"]):
    prd_sample_len.append(len(prd))
    tgr_sample_len.append(len(tgr))

plot_scatter_with_dist(prd_sample_len, tgr_sample_len, "Prediction & target length", "Predic. length", "Target length")


### Analyze use of indices

In [ ]:
global_c = 0
global_f = 0
other_c = 0
other_f = 0

for src, prd in tqdm(list(zip(model_d.get("source_sentences"), model_d.get("predic_sentences")))):
    one_line_src = src.replace("\n", " ")
    is_global = one_line_src.find("__global__") != -1
    
    prd_lines = prd.splitlines()
    found = False
    for line in prd_lines:
        if line.find("threadIdx.") != -1 and line.find("blockIdx.") != -1 and line.find("blockDim.") != -1:
            if is_global:
                global_c += 1
            else:
                other_f += 1
            found = True
            break
        
    if not found:
        if is_global:
            global_f += 1
        else:
            other_c += 1

print(f"global_c: {global_c} ({global_c/(global_c+global_f):.2%})")
print(f"global_f: {global_f} ({global_f/(global_c+global_f):.2%})")
print(f"others_c: {other_c} ({other_c/(other_c+other_f):.2%})")
print(f"others_f: {other_f} ({other_f/(other_c+other_f):.2%})")

total = global_c + global_f + other_c + other_f
correct = global_c + other_c
incorrect = global_f + other_f
print(f"Correct: {correct} ({correct/total:.2%})")
print(f"Incorrect: {incorrect}, ({incorrect/total:.2%})")

### Use of local memory analyses

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
t_p = 0
t_n = 0
f_p = 0
f_n = 0

t = []
p = []

for tgr, prd in tqdm(list(zip(model_d.get("target_sentences"), model_d.get("predic_sentences")))):
    tgr_l_m_use = tgr.find("__shared__") != -1 or tgr.find("__constant__") != -1
    prd_l_m_use = prd.find("__shared__") != -1 or prd.find("__constant__") != -1
    
    if tgr_l_m_use:
        if prd_l_m_use:
            t_p += 1
        else:
            f_n += 1
    else:
        if prd_l_m_use:
            f_p += 1
        else:
            t_n += 1
            
    t.append(tgr_l_m_use)
    p.append(prd_l_m_use)
    
total = t_p + t_n + f_p + f_n
print(f"True positive: {t_p}\t({t_p/total:.2%})")
print(f"True negative: {t_n}\t({t_n/total:.2%})")
print(f"False positive: {f_p}\t({f_p/total:.2%})")
print(f"False negative: {f_n}\t({f_n/total:.2%})")

precision = precision_score(t, p)
recall = recall_score(t, p)
f1 = f1_score(t, p)

print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1: {f1:.3f}")